## Data Merging (1)

### Data merging basics

여러개의 table을 하나의 table을 합치고 싶을 수 있다.  

In [24]:
import pandas as pd
import numpy as np
import pickle

taxi_veh = pd.read_pickle('taxi_vehicles.p')
taxi_owners = pd.read_pickle('taxi_owners.p')
wards = pd.read_pickle('ward.p')
census = pd.read_pickle('census.p')
licenses = pd.read_pickle('licenses.p')
biz_owners = pd.read_pickle('business_owners.p')
zip_demo = pd.read_pickle("zip_demo.p")
land_use = pd.read_pickle('land_use.p')
movies = pd.read_pickle('movies.p')
financials = pd.read_pickle('financials.p')
movie_to_genres = pd.read_pickle('movie_to_genres.p')
taglines = pd.read_pickle('taglines.p')

print(taxi_veh.head())
print()
print(taxi_owners.head())

    vid    make   model  year fuel_type                owner
0  2767  TOYOTA   CAMRY  2013    HYBRID       SEYED M. BADRI
1  1411  TOYOTA    RAV4  2017    HYBRID          DESZY CORP.
2  6500  NISSAN  SENTRA  2019  GASOLINE       AGAPH CAB CORP
3  2746  TOYOTA   CAMRY  2013    HYBRID  MIDWEST CAB CO, INC
4  5922  TOYOTA   CAMRY  2013    HYBRID       SUMETTI CAB CO

     rid   vid           owner                 address    zip
0  T6285  6285  AGEAN TAXI LLC     4536 N. ELSTON AVE.  60630
1  T4862  4862    MANGIB CORP.  5717 N. WASHTENAW AVE.  60659
2  T1495  1495   FUNRIDE, INC.     3351 W. ADDISON ST.  60618
3  T4231  4231    ALQUSH CORP.   6611 N. CAMPBELL AVE.  60645
4  T5971  5971  EUNIFFORD INC.     3351 W. ADDISON ST.  60618


두 table에 `vid`로 묶을 수 있을 것 같다. 이는 다음과 같이 수행할 수 있다. 이런식으로 두 table에 있는 `vid`에 대해 match하는 value를 가진 rows를 반환하는 것을 inner join이라 한다.  

In [25]:
taxi_own_veh = taxi_owners.merge(taxi_veh,on="vid")
print(taxi_own_veh.columns)

Index(['rid', 'vid', 'owner_x', 'address', 'zip', 'make', 'model', 'year',
       'fuel_type', 'owner_y'],
      dtype='object')


각각에 `owner`가 있어 `_x`,`_y` 이름 붙은 것을 확인 할 수 있다. 이를 보기 쉽게 하기 위해, `_own`, `_veh` 로 바꿔보자.  

In [26]:
taxi_own_veh = taxi_owners.merge(taxi_veh,on="vid", suffixes= ('_own','_veh'))
print(taxi_own_veh.columns)

Index(['rid', 'vid', 'owner_own', 'address', 'zip', 'make', 'model', 'year',
       'fuel_type', 'owner_veh'],
      dtype='object')


merge된 table을 가지고 어떤 `fuel_type`이 많은 지 확인 해보자.  

In [27]:
print(taxi_own_veh['fuel_type'].value_counts())

HYBRID                    2792
GASOLINE                   611
FLEX FUEL                   89
COMPRESSED NATURAL GAS      27
Name: fuel_type, dtype: int64


`licenses`와 `biz_owners`를 `account`로 merge 한 것을 `licenses_owners`라 하자.  
`counted_df`는 `licenses_owners`를 `title`로 group하고 각각의 `title`에 대해 account가 몇 번 나왔는지를 세는 거다.  
그리고 `account`를 내림차순으로 `counted_df`를 sort 한 것을 `sorted_df`라 하자.  

In [28]:
licenses_owners = licenses.merge(biz_owners, on="account")
counted_df = licenses_owners.groupby("title").agg({'account':'count'})
sorted_df = counted_df.sort_values("account",ascending = False)

print(sorted_df.head())

                 account
title                   
PRESIDENT           6259
SECRETARY           5205
SOLE PROPRIETOR     1658
OTHER               1200
VICE PRESIDENT       970


In [29]:
print(licenses_owners.head())

  account ward  aid              business              address    zip  \
0  307071    3  743  REGGIE'S BAR & GRILL      2105 S STATE ST  60616   
1      10   10  829            HONEYBEERS  13200 S HOUSTON AVE  60633   
2      10   10  829            HONEYBEERS  13200 S HOUSTON AVE  60633   
3   10002   14  775           CELINA DELI    5089 S ARCHER AVE  60632   
4   10002   14  775           CELINA DELI    5089 S ARCHER AVE  60632   

  first_name last_name      title  
0     ROBERT     GLICK     MEMBER  
1      PEARL   SHERMAN  PRESIDENT  
2      PEARL   SHERMAN  SECRETARY  
3     WALTER    MROZEK    PARTNER  
4     CELINA    BYRDAK    PARTNER  


2개 이상의 table을 merge 할 수도 있다.  

In [30]:
licenses_zip_ward = licenses.merge(zip_demo, on = "zip") \
            			.merge(wards, on = "ward")
print((licenses_zip_ward).groupby("alderman").agg({'income':'median'}).head())

                         income
alderman                       
Ameya Pawar             66246.0
Anthony A. Beale        38206.0
Anthony V. Napolitano   82226.0
Ariel E. Reyboras       41307.0
Brendan Reilly         110215.0


`\`은 뒤의 코드가 이어진다는 뜻이다.  

left join은 쉽게 말해 왼쪽 거는 살리고 오른쪽 거는 날리는 거다.  `how`를 통해 방법을 설정하는 것이 가능하다.  

In [31]:
movies_financials = movies.merge(financials, on = "id", how = 'left')
print(movies_financials.head())

      id                 title  popularity release_date      budget  \
0    257          Oliver Twist   20.415572   2005-09-23  50000000.0   
1  14290  Better Luck Tomorrow    3.877036   2002-01-12         NaN   
2  38365             Grown Ups   38.864027   2010-06-24  80000000.0   
3   9672              Infamous    3.680896   2006-11-16  13000000.0   
4  12819       Alpha and Omega   12.300789   2010-09-17  20000000.0   

       revenue  
0   42093706.0  
1          NaN  
2  271430189.0  
3    1151330.0  
4   39300000.0  


In [32]:
number_of_missing_fin = movies_financials['budget'].isnull().sum()
print(number_of_missing_fin)

1574


In [35]:
print(movies)

         id                 title  popularity release_date
0       257          Oliver Twist   20.415572   2005-09-23
1     14290  Better Luck Tomorrow    3.877036   2002-01-12
2     38365             Grown Ups   38.864027   2010-06-24
3      9672              Infamous    3.680896   2006-11-16
4     12819       Alpha and Omega   12.300789   2010-09-17
...     ...                   ...         ...          ...
4798   3089             Red River    5.344815   1948-08-26
4799  11934   The Hudsucker Proxy   14.188982   1994-03-11
4800  13807                Exiled    8.486390   2006-09-06
4801  73873          Albert Nobbs    7.802245   2011-12-21
4802  11622   Blast from the Past    8.737058   1999-02-12

[4803 rows x 4 columns]
